# A Step-by-Step Guide to HoloFusion example part 1/4
  # (majority vote)


# Setup
Firstly, we import all the module from Holofusion that we will use.

In [1]:
from holofusion import HoloFusion, HoloFusionSession
from time import time as t

##   Initialization
In this part, we create the HoloFusion and Session object that we will use for this example.

In [2]:
        holo_obj = HoloFusion(algorithm=1, training_data=0, multiple_weights=0)
        session = HoloFusionSession("Session", holo_obj)
        fx = open('execution_time.txt', 'w')
        

/home/james/anaconda/envs/python2.7/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1576: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input from file
Test data will be read using the Session's ingestor.
After ingesting the test data will be loaded into MySQL tables along with entries in the a metadata table.

In [3]:
        list_time = []
        start_time = t()

        session.ingest_dataset("data/clean_flight2/flight-data-test.csv")
        d = t()-start_time
        list_time.append(d)
        fx.write('ingest csv time: '+str(d)+'\n')
        print 'ingest csv time: '+str(d)+'\n'
        
        start_time = t()
        session.adding_training_data("data/clean_flight2/flight-data_training.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('adding training data: ' + str(d) + '\n')
        print 'adding training time: ' + str(d) + '\n'
        
        print 'Clean table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_clean", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()
        print 'Don\'t know table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_dk", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()

ingest csv time: 7.89807105064

adding training time: 7.13273978233

Clean table
+------+----------+--------------+-----------+
|Source|Flight_Num|Scheduled_Dept|Actual_Dept|
+------+----------+--------------+-----------+
+------+----------+--------------+-----------+

Don't know table
+-------------------+--------------------+--------------+--------------------+
|             Source|          Flight_Num|Scheduled_Dept|         Actual_Dept|
+-------------------+--------------------+--------------+--------------------+
|                 aa|2011-12-01-AA-100...|          1:55|                2:07|
|     flightexplorer|2011-12-01-AA-100...|          null|                2:07|
|    airtravelcenter|2011-12-01-AA-100...|          null|                5:00|
|         myrateplan|2011-12-01-AA-100...|          null|                5:00|
|        helloflight|2011-12-01-AA-100...|          null|                5:00|
|          flytecomm|2011-12-01-AA-100...|          null|                5:00|
| 

## Featurization

In [4]:
        start_time = t()
        session.feature()
        d = t() - start_time
        list_time.append(d)
        fx.write('creating feature table time: '+str(d)+'\n')
        print 'Feature table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Feature", session.dataset)
        sql.show()
        print ' feature table time: '+str(d)+'\n'

adding weight_id to feature table...
creating weight table
creating feature table with weights ids
adding weight_id to feature table is finished
Feature table
+---------+-------------------+--------------------+-----------------+------------+---------+-----+
|var_index|          Source_id|            rv_index|          rv_attr|assigned_val|weight_id|fixed|
+---------+-------------------+--------------------+-----------------+------------+---------+-----+
|        1|                 aa|2011-12-01-AA-100...|   Actual_Arrival|        2:57|        1|    0|
|        2|     flightexplorer|2011-12-01-AA-100...|   Actual_Arrival|        2:51|       30|    0|
|        3|    airtravelcenter|2011-12-01-AA-100...|   Actual_Arrival|        5:00|       31|    0|
|        4|         myrateplan|2011-12-01-AA-100...|   Actual_Arrival|        5:00|       14|    0|
|        5|        helloflight|2011-12-01-AA-100...|   Actual_Arrival|        5:00|       15|    0|
|        6|          flytecomm|2011-12-01

#  Learning
In the learning phase, for majority vote we just assume that each weight takes the value 1.


In [5]:
        start_time = t()
        session.inference()
        d = t() - start_time
        list_time.append(d)
        fx.write('inference time: ' + str(d) + '\n')
        print 'Probabilities table'
        sql = holo_obj.dataengine.get_table_to_dataframe("Probabilities", session.dataset)
        sql.show()
        print 'inference time: ' + str(d) + '\n'

Creating Group by Accur
Creating Probability table
Creating Final table
Probabilities table
+--------------------+-----------------+------------+--------------------+
|            rv_index|          rv_attr|assigned_val|         probability|
+--------------------+-----------------+------------+--------------------+
|2011-12-01-AA-100...|   Actual_Arrival|        2:51|0.024288897679263205|
|2011-12-01-AA-100...|   Actual_Arrival|        2:57| 0.48785555116036833|
|2011-12-01-AA-100...|   Actual_Arrival|        5:00| 0.48785555116036833|
|2011-12-01-AA-100...|      Actual_Dept|        2:07|  0.7310585786300049|
|2011-12-01-AA-100...|      Actual_Dept|        5:00|  0.2689414213699951|
|2011-12-01-AA-100...|Scheduled_Arrival|        3:00|                 1.0|
|2011-12-01-AA-100...|   Scheduled_Dept|        1:55|                 1.0|
|2011-12-01-AA-101...|   Actual_Arrival|        5:00|  0.7310585786300048|
|2011-12-01-AA-101...|   Actual_Arrival|        8:36|  0.2689414213699951|
|2011-12

# Evaluation
 In this part, we find the accuracy of our results by comparing them to ground truth for our data

In [6]:
        start_time = t()
        session.accuracy("data/clean_flight2/flight-data_truth.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('time to calculate accuracy: ' + str(d) + '\n')
        print 'time to calculate accuracy: ' + str(d) + '\n'
        fx.close()

['Flight_Num', 'Scheduled_Dept', 'Actual_Dept', 'Scheduled_Arrival', 'Actual_Arrival']
show the final table
+--------------------+-----------------+------------+
|            rv_index|          rv_attr|assigned_val|
+--------------------+-----------------+------------+
|2011-12-01-AA-100...|   Actual_Arrival|        5:00|
|2011-12-01-AA-100...|      Actual_Dept|        2:07|
|2011-12-01-AA-100...|Scheduled_Arrival|        3:00|
|2011-12-01-AA-100...|   Scheduled_Dept|        1:55|
|2011-12-01-AA-101...|   Actual_Arrival|        5:00|
|2011-12-01-AA-101...|      Actual_Dept|        5:00|
|2011-12-01-AA-101...|Scheduled_Arrival|        8:40|
|2011-12-01-AA-101...|   Scheduled_Dept|        7:15|
|2011-12-01-AA-103...|   Actual_Arrival|        5:00|
|2011-12-01-AA-103...|      Actual_Dept|        5:00|
|2011-12-01-AA-103...|Scheduled_Arrival|        3:20|
|2011-12-01-AA-103...|   Scheduled_Dept|        1:25|
|2011-12-01-AA-104...|   Actual_Arrival|        1:17|
|2011-12-01-AA-104...|      